### Machine Learning is defined as:
* Field of study where we give machines the ability to learn without being explicitly programmed to do so.
* In viewing knowledge to a machine without hardcoding that knowledge.

In [55]:
import pandas as pd
import quandl, math
import numpy as np
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, train_test_split

In [7]:
df = quandl.get('WIKI/GOOGL') 

In [8]:
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2004-08-19,100.01,104.06,95.96,100.335,44659000.0,0.0,1.0,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,101.01,109.08,100.50,108.310,22834300.0,0.0,1.0,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,110.76,113.48,109.05,109.400,18256100.0,0.0,1.0,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,111.24,111.60,103.57,104.870,15247300.0,0.0,1.0,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,104.76,108.00,103.88,106.000,9188600.0,0.0,1.0,52.542193,54.167209,52.100830,53.164113,9188600.0


In [10]:
df = df.iloc[:,7:12]

In [11]:
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0


In [14]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. High'] * 100

In [15]:
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT
Date,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,3.579666
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,0.705904
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,3.595347
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,6.030466
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,1.851852


In [16]:
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100

In [17]:
df.head()

,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume,HL_PCT,PCT_change
Date,,,,,,,
2004-08-19,50.159839,52.191109,48.128568,50.322842,44659000.0,3.579666,0.324968
2004-08-20,50.661387,54.708881,50.405597,54.322689,22834300.0,0.705904,7.227007
2004-08-23,55.551482,56.915693,54.693835,54.869377,18256100.0,3.595347,-1.227880
2004-08-24,55.792225,55.972783,51.945350,52.597363,15247300.0,6.030466,-5.726357
2004-08-25,52.542193,54.167209,52.100830,53.164113,9188600.0,1.851852,1.183658


In [20]:
df = df.iloc[:,3:]

In [21]:
df.head()

,Adj. Close,Adj. Volume,HL_PCT,PCT_change
Date,,,,
2004-08-19,50.322842,44659000.0,3.579666,0.324968
2004-08-20,54.322689,22834300.0,0.705904,7.227007
2004-08-23,54.869377,18256100.0,3.595347,-1.227880
2004-08-24,52.597363,15247300.0,6.030466,-5.726357
2004-08-25,53.164113,9188600.0,1.851852,1.183658


In [22]:
forecast_col = 'Adj. Close'

In [23]:
df.fillna(-99999,inplace = True)

In [88]:
forecast_out = int(math.ceil((0.01*len(df))))
forecast_out

31

In [39]:
df['label'] = df[forecast_col].shift(forecast_out)

In [40]:
df.dropna(inplace = True)

In [41]:
df.head()

,Adj. Close,Adj. Volume,HL_PCT,PCT_change,label
Date,,,,,
2006-02-13,173.385224,39396100.0,1.397604,0.406622,214.005615
2006-02-14,172.191539,29278600.0,2.379937,-0.573414,210.725489
2006-02-15,171.720083,25868100.0,1.046243,0.287053,208.072299
2006-02-16,183.797365,42588300.0,0.147139,5.956167,218.288837
2006-02-17,184.945910,28611700.0,0.910947,-0.335144,223.309335


### Goal is to keep the features between 1 and -1 this can help in getting good accuracy and processing speed.
* **Features** is represented by capital **X**.
* **labels** is represented by small **y**.

In [65]:
X = np.array(df.drop(['label'],1))

In [66]:
y = np.array(df['label'])

* Now we will scale **X**.

In [67]:
X = preprocessing.scale(X)

In [68]:
y = np.array(df['label'])

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [89]:
# cls = LinearRegression()
cls = svm.SVR()
# cls = svm.SVR(kernel = 'poly')

In [90]:
cls.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [95]:
accuracy = cls.score(X_test, y_test)

In [96]:
accuracy

0.7716629834036982